In [1]:
import struct
import numpy as np
from PIL import Image
from pathlib import Path
import os
import shutil
import traceback
import sys
import geopandas as gpd
from tqdm import tqdm
import subprocess
import re
from pyproj import transform, Proj
from shapely import geometry, affinity
from rasterio.control import GroundControlPoint
from rasterio.transform import from_gcps
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from osgeo import gdal, osr; gdal.UseExceptions()
import pickle
from IPython.display import clear_output
import warnings; warnings.filterwarnings("ignore")


In [2]:
distance = 25
humidity = 90
emissivity = 0.95
reflection = 0

In [3]:
# directory structure prparation
rjpg_dir = 'data/input/rjpg'
tiff_dir = 'data/output/tiff'
geotiff_dir = 'data/output/geotiff'
footprints_path = "data/output/footprints.geojson"
mosaic_path = "data/output/mosaic.tiff"
tmp_dir = 'data/tmp'
if os.path.exists(tmp_dir):
    shutil.rmtree(tmp_dir)
os.mkdir(tmp_dir)

# R-JPG to TIFF

In [4]:
distance = round(distance)
humidity = round(humidity)
emissivity = round(emissivity,2)
reflection = round(reflection)

assert os.path.exists(rjpg_dir)
if os.path.exists(tiff_dir):
    shutil.rmtree(tiff_dir)
os.makedirs(tiff_dir)
# get list of .jpg files from input directory
file_list = list(filter(lambda x:x.endswith((".jpg", ".JPG")), os.listdir(rjpg_dir)))
# iterate over files
for i, file_name in enumerate(file_list):
    # get temporary raw byte file by DJI Thermal SDK
    #clear cell output
    clear_output(wait=True)
    print(f"File {i+1}/{len(file_list)}")
    os.system(  f"./dji_irp -a measure \
                --measurefmt float32 \
                --distance {distance} \
                --humidity {humidity} \
                --emissivity {emissivity} \
                --reflection {reflection} \
                -s {rjpg_dir}/{file_name} -o {tmp_dir}/bytes.raw")
    # get image size
    img = Image.open(f"{rjpg_dir}/{file_name}")
    # decode temporary byte file to tiff
    arr = np.zeros(img.size[0]*img.size[1])
    with open(f"{tmp_dir}/bytes.raw", "rb") as f:
        data = f.read()
        format = '{:d}f'.format(len(data)//4)
        arr=np.array(struct.unpack(format, data))
    arr = arr.reshape(img.size[1],img.size[0])
    im = Image.fromarray(arr)
    im.save(f"{tiff_dir}/{file_name}.tiff")
    # copy exif data from original file to new tiff
    os.system(f"exiftool -tagsfromfile {rjpg_dir}/{file_name} {tiff_dir}/{file_name}.tiff -overwrite_original_in_place")

File 453/453
DIRP API version number : 0x13
DIRP API magic version  : d4c7dea
R-JPEG file path : data/input/rjpg/DJI_20220324155129_0453_T.JPG
R-JPEG version information
    R-JPEG version : 0x1
    header version : 0x103
 curve LUT version : 0x1
R-JPEG resolution size
      image  width : 640
      image height : 512
Change distance from 5 to 25
Change humidity from 70 to 90
Change emissivity from 1 to 0.95
Change reflection from 23 to 0
Measurement: get params range:
distance: [1,25]
humidity: [20,100]
emissivity: [0.1,1]
reflection: [-40,500]
Run action 1
Save image file as : data/tmp/bytes.raw
Test done with return code 0
    1 image files updated


# TIFF to GeoTIFF

In [5]:
assert os.path.exists(tiff_dir)
if os.path.exists(geotiff_dir):
    shutil.rmtree(geotiff_dir)
os.makedirs(geotiff_dir)
if os.path.exists(footprints_path):
    os.remove(footprints_path)

def dms2dec(dms):
    d = float(dms[0])
    m = float(dms[1])
    s = float(dms[2])
    return (d+m/60+s/3600)*(-1 if dms[3] in ['W', 'S'] else 1)

print("Georeferencing photos...")
dfov_d = 40 #degrees
hcoef = 640/819.5999023914046
vcoef = 512/819.5999023914046
names = sorted([name for name in os.listdir(tiff_dir) if name.endswith(".tiff")])
footprints = gpd.GeoDataFrame(columns=["name", "geometry"])
for name in tqdm(names):
    file_path = f"{tiff_dir}/{name}"
    # get exif
    exif = {}
    lat, lon, yaw, alt = subprocess.run(['exiftool', '-GPSLatitude', '-GPSLongitude', '-GimbalYawDegree', '-RelativeAltitude', file_path], stdout=subprocess.PIPE).stdout.decode("utf-8").splitlines()
    lat = lat.split(":",1)[1].strip()
    lat = [x.strip() for x in re.split('[deg\'"]', lat) if x]
    lat = dms2dec(lat)
    exif['lat'] = lat
    lon = lon.split(":",1)[1].strip()
    lon = [x.strip() for x in re.split('[deg\'"]', lon) if x]
    lon = dms2dec(lon)
    exif['lon'] = lon
    yaw = yaw.split(":",1)[1].strip()
    exif['yaw'] = float(yaw)
    alt = alt.split(":",1)[1].strip()
    exif['alt'] = float(alt)
    #calculate polygon FOV
    dfov_m = 2*exif['alt']*np.tan(dfov_d/2*np.pi/180)
    vfov_m = vcoef*dfov_m
    hfov_m = hcoef*dfov_m
    x, y = transform(Proj(init='epsg:4326'), Proj(init='epsg:32634'), exif['lon'], exif['lat'])
    polygon = geometry.Polygon([(x-hfov_m/2, y-vfov_m/2), (x+hfov_m/2, y-vfov_m/2), (x+hfov_m/2, y+vfov_m/2), (x-hfov_m/2, y+vfov_m/2)])
    polygon = affinity.rotate(polygon,-exif['yaw'], origin="centroid")
    footprints = footprints.append({"name": name, "geometry": polygon}, ignore_index=True)
    #convert tiff to geotiff using FOV
    crs = 'epsg:32634'
    raster = rxr.open_rasterio(f"{tiff_dir}/{name}")
    arr = raster.values.squeeze()
    tl = GroundControlPoint(0, 0, polygon.exterior.coords[3][0], polygon.exterior.coords[3][1])
    tr = GroundControlPoint(0, arr.shape[1], polygon.exterior.coords[2][0], polygon.exterior.coords[2][1])
    br = GroundControlPoint(arr.shape[0], arr.shape[1], polygon.exterior.coords[1][0], polygon.exterior.coords[1][1])
    bl = GroundControlPoint(arr.shape[0], 0, polygon.exterior.coords[0][0], polygon.exterior.coords[0][1])
    gcps = [tl, bl, br, tr]
    trans = from_gcps(gcps)
    raster = raster.rio.write_transform(trans)
    raster = raster.rio.write_crs(crs)
    raster.rio.to_raster(f"{tmp_dir}/raster.tiff", driver='GTiff')
    gdal.Warp(f"{geotiff_dir}/{name}", f"{tmp_dir}/raster.tiff", format='GTiff', dstSRS=crs, dstNodata=-9999)
footprints = footprints.set_crs('epsg:32634')
footprints.to_file(footprints_path, driver="GeoJSON")

Georeferencing photos...


100%|██████████| 453/453 [03:01<00:00,  2.50it/s]


# GeoTIFF photos to mosaic

In [6]:
rasters = []
print("Loading...")
for name in tqdm(names):
    rasters.append(rxr.open_rasterio(os.path.join(geotiff_dir, name)))
print("Merging...")
mosaic = merge_arrays(rasters)
mosaic.rio.to_raster(mosaic_path, driver='GTiff')

Loading...


100%|██████████| 453/453 [00:07<00:00, 60.67it/s]


Merging...


# Cleanup

In [7]:
#clear temporary files
shutil.rmtree(tmp_dir)